In [52]:
import requests
import requests.packages.urllib3.util.connection as urllib3_cn
import socket
import bs4
import csv
from sentence_transformers import SentenceTransformer
import pandas as pd


def address_family():
    return socket.AF_INET


urllib3_cn.allowed_gai_family = address_family


In [53]:
alph = 'a'


def cdc_page_url(alphabet):
    return 'https://www.cdc.gov/diseasesconditions/az/' + alphabet + '.html'


disease_list = []

while alph != 'd':
    page_response = requests.get(cdc_page_url(alph))
    soup = bs4.BeautifulSoup(page_response.text, 'html.parser')
    list = soup.find('div', {'class': 'az-content'}
                     ).findChild('ul').findChildren('li', recursive=False)
    for item in list:
        disease_list.append(item.findChild('a').getText())
    alph = chr(ord(alph) + 1)


In [54]:
with open('disease_list.csv', 'w') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['disease'])
    writer.writeheader()
    for disease in disease_list:
        writer.writerow({'disease': disease})
    csv_file.close()

diseases_df = pd.read_csv('disease_list.csv')


def remove_references(x):
    return x.split('see')[0]


def remove_appendations(x):
    if '-' in x:
        return x.split('-')[0]
    if '—' in x:
        return x.split('—')[0]
    else:
        return x


def clean_parentheses(x):
    if '(' not in x:
        return x
    else:
        cleaned_up = ''
        opening_split = x.split('(')
        for item in opening_split:
            closing_split = item.split(')')
            for phrase in closing_split:
                if len(phrase) > len(cleaned_up):
                    cleaned_up = phrase
        return cleaned_up


def clean_square_brackets(x):
    if '[' not in x:
        return x
    else:
        cleaned_up = ''
        opening_split = x.split('[')
        for item in opening_split:
            closing_split = item.split(']')
            for phrase in closing_split:
                if len(phrase) > len(cleaned_up):
                    cleaned_up = phrase
        return cleaned_up


diseases_df.disease = diseases_df.disease.apply(lambda x: x.lower())
diseases_df.disease = diseases_df.disease.apply(lambda x: remove_references(x))
diseases_df.disease = diseases_df.disease.apply(
    lambda x: remove_appendations(x))
diseases_df.disease = diseases_df.disease.apply(lambda x: clean_parentheses(x))
diseases_df.disease = diseases_df.disease.apply(
    lambda x: clean_square_brackets(x))
diseases_df.head(20)


,disease
0,abdominal aortic aneurysm
1,acanthamoeba infection
2,adverse childhood experiences
3,acinetobacter infection
4,acquired immune deficiency syndrome
5,acute flaccid myelitis
6,adenovirus infection
7,adenovirus vaccination
8,attention deficit/hyperactivity disorder
9,adult vaccinations
